In [1]:
from pathlib import Path
import json
import pandas as pd

LOG_PATH = Path("/home/moon/Desktop/Financial_Document_Analyzer/data/logs/financebench_eval_20251125_103847.jsonl")
assert LOG_PATH.exists(), f"Log file not found: {LOG_PATH}"

rows = []
with LOG_PATH.open("r", encoding="utf-8") as fh:
    for line in fh:
        line = line.strip()
        if not line:
            continue
        rows.append(json.loads(line))
df = pd.DataFrame(rows)
df

,doc_name,question,ground_truth,question_type,question_reasoning,answer,citations,hits,eval_is_same,eval_reasoning
0,AMERICANEXPRESS_2022_10K,Which debt securities are registered to trade ...,There are none,domain-relevant,Information extraction,The provided text appears to be excerpts from ...,[],"[{'chunk_id': '943', 'source_doc': 'AMERICANEX...",False,None
1,AMERICANEXPRESS_2022_10K,What are the geographies that American Express...,"United States, EMEA, APAC, and LACC",domain-relevant,Information extraction,The geographies that American Express primaril...,"[{'i': 3, 'source_doc': 'AMERICANEXPRESS_2022_...","[{'chunk_id': '1197', 'source_doc': 'AMERICANE...",False,None
2,AMERICANEXPRESS_2022_10K,Does AMEX have an improving operating margin p...,Performance is not measured through operating ...,domain-relevant,Numerical reasoning OR information extraction,American Express,[],"[{'chunk_id': '492', 'source_doc': 'AMERICANEX...",False,None
3,AMERICANEXPRESS_2022_10K,What drove gross margin change as of the FY202...,Performance is not measured through gross margin,domain-relevant,Logical reasoning (based on numerical reasonin...,No answer,"[{'i': 1, 'source_doc': 'AMERICANEXPRESS_2022_...","[{'chunk_id': '492', 'source_doc': 'AMERICANEX...",True,"Both answers convey the same meaning, indicati..."
4,AMERICANEXPRESS_2022_10K,How much has the effective tax rate of America...,The effective tax rate for American Express ha...,domain-relevant,Numerical reasoning,The effective tax rate decreased by 3 percenta...,"[{'i': 1, 'source_doc': 'AMERICANEXPRESS_2022_...","[{'chunk_id': '275', 'source_doc': 'AMERICANEX...",False,None
5,AMERICANEXPRESS_2022_10K,What was the largest liability in American Exp...,Customer deposits,novel-generated,None,No answer,[],"[{'chunk_id': '24', 'source_doc': 'AMERICANEXP...",False,None
6,AMERICANEXPRESS_2022_10K,Was American Express able to retain card membe...,Yes,novel-generated,None,Yes.,[],"[{'chunk_id': '24', 'source_doc': 'AMERICANEXP...",True,None


In [2]:
# Basic metrics
total = len(df)
correct = int(df['eval_is_same'].sum())
accuracy = correct / total if total else 0
print(f'Total questions: {total}')
print(f'Correct: {correct} ({accuracy:.2%})')

print('\nAccuracy by question_type:')
grouped = df.groupby('question_type')['eval_is_same'].agg(['mean', 'count'])
display(grouped.rename(columns={'mean': 'accuracy'}))


Total questions: 7
Correct: 2 (28.57%)

Accuracy by question_type:


,accuracy,count
question_type,,
domain-relevant,0.2,5
novel-generated,0.5,2


In [3]:
# Inspect incorrect predictions
bad_cases = df[df['eval_is_same'] == False]
print(f'Bad cases: {len(bad_cases)}')
bad_cases[['question', 'question_type', 'answer', 'ground_truth']]


Bad cases: 5


,question,question_type,answer,ground_truth
0,Which debt securities are registered to trade ...,domain-relevant,The provided text appears to be excerpts from ...,There are none
1,What are the geographies that American Express...,domain-relevant,The geographies that American Express primaril...,"United States, EMEA, APAC, and LACC"
2,Does AMEX have an improving operating margin p...,domain-relevant,American Express,Performance is not measured through operating ...
4,How much has the effective tax rate of America...,domain-relevant,The effective tax rate decreased by 3 percenta...,The effective tax rate for American Express ha...
5,What was the largest liability in American Exp...,novel-generated,No answer,Customer deposits


In [4]:
# Inspect a specific bad case (default: first failure)
idx = bad_cases.index[0] if len(bad_cases) else None
if idx is not None:
    rec = bad_cases.loc[idx]
    print('Question:', rec['question'])
    print('\nGround Truth:', rec['ground_truth'])
    print('\nModel Answer:', rec['answer'])
    print('\nCitations:', rec['citations'])
    print('\nHits (top-k chunk excerpts):')
    for hit in rec['hits']:
        print('-', hit.get('doc_id'), '|', (hit.get('text') or ''))
else:
    print('No bad cases available.')


Question: Which debt securities are registered to trade on a national securities exchange under American Express' name as of 2022?

Ground Truth: There are none

Model Answer: The provided text appears to be excerpts from a financial report, likely an annual report for American Express Company. The main topics discussed are the company's capital ratios and its relationship with securitization trusts.

Citations: []

Hits (top-k chunk excerpts):
- AMERICANEXPRESS_2022_10K_p176 | AMERICAN EXPRESS COMPANY AMERICAN EXPRESS NATIONAL BANK
- AMERICANEXPRESS_2022_10K_p252 | American Express Company 56th Street AXP Campus LLC American Express Bank LLC Russian Federation American Express Banking Corp. American Express Travel Related Services Company, Inc. Netherlands Antilles Page 1
- AMERICANEXPRESS_2022_10K_p252 | Country Name Jurisdiction United States New York United States Arizona Russian Federation United States New York United States New York Accertify, Inc. United States Delaware AE Inno

In [5]:
# Inspect correct predictions
good_cases = df[df['eval_is_same'] == True]
print(f'Good cases: {len(good_cases)}')
good_cases[['question', 'question_type', 'answer', 'ground_truth']]

Good cases: 2


,question,question_type,answer,ground_truth
3,What drove gross margin change as of the FY202...,domain-relevant,No answer,Performance is not measured through gross margin
6,Was American Express able to retain card membe...,novel-generated,Yes.,Yes


In [7]:
# Inspect a specific good case (default: first failure)
idx = good_cases.index[1] if len(good_cases) else None
if idx is not None:
    rec = good_cases.loc[idx]
    print('Question:', rec['question'])
    print('\nGround Truth:', rec['ground_truth'])
    print('\nModel Answer:', rec['answer'])
    print('\nCitations:', rec['citations'])
    print('\nHits (top-k chunk excerpts):')
    for hit in rec['hits']:
        print("'chunk_id:'", hit.get('chunk_id'), '|', (hit.get('text') or ''))
else:
    print('No bad cases available.')


Question: Was American Express able to retain card members during 2022?

Ground Truth: Yes

Model Answer: Yes.

Citations: []

Hits (top-k chunk excerpts):
'chunk_id:' 24 | For the year ended December 31, 2022, worldwide billed business (spending on American Express cards issued by us) was $1,338 billion and at December 31, 2022, we had 76.7 million proprietary cards in force worldwide.
'chunk_id:' 5 | ° American Express Company
'chunk_id:' 36 | The American Express Brand and Service Excellence
'chunk_id:' 491 | To the Board of Directors and Shareholders of American Express Company
'chunk_id:' 165 | Revenue growth is dependent on increasing consumer and business spending on our cards, growing loan balances and increasing fee revenue. We have been investing in a number of growth initiatives, including to attract new Card Members, retain existing Card Members and capture a greater share of customers’ total spending and borrowings. There can be no assurance that our investments will conti